In [57]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [58]:
sig_bound = (0.05, 2.5)
ep_bound = (0.001, 0.5)
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [59]:
ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [60]:
# Parameters
# ffpkl = "mdcm_pbe0dz_FOE.ff.pkl"


#  LJ fit


In [61]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c

FF: LJ water_cluster ECOL Exact first_order_energy (jax_coloumb: False)

In [62]:
# _c.targets

In [63]:
fit_repeat(_c, 
           10, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           # quiet='true'
          )

Optimizing LJ parameters...
function: get_loss_jax
bounds: ((0.05, 2.5), (0.05, 2.5), (0.001, 0.5), (0.001, 0.5))
maxfev: 10000
initial guess: [1.5598225295554273, 2.054675399618392, 0.35806922121640306, 0.12302384436255726]


/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


final_loss_fn:  41.232906341552734
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 41.232906341552734
             x: [ 5.000e-02  2.500e+00  9.906e-02  7.106e-03]
           nit: 379
          nfev: 670
 final_simplex: (array([[ 5.000e-02,  2.500e+00,  9.906e-02,  7.106e-03],
                       [ 5.000e-02,  2.500e+00,  9.906e-02,  7.106e-03],
                       ...,
                       [ 5.000e-02,  2.500e+00,  9.906e-02,  7.106e-03],
                       [ 5.000e-02,  2.500e+00,  9.906e-02,  7.106e-03]]), array([ 4.123e+01,  4.123e+01,  4.123e+01,  4.123e+01,
                        4.123e+01]))
Optimizing LJ parameters...
function: get_loss_jax
bounds: ((0.05, 2.5), (0.05, 2.5), (0.001, 0.5), (0.001, 0.5))
maxfev: 10000
initial guess: [0.6976780759883263, 2.073661459321012, 0.03144216842979983, 0.3893133574855197]
final_loss_fn:  45.203372955322266
       message: Optimization terminated successfully.
       

In [68]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
6,19.146339,850,1444,0,True,Optimization terminated successfully.,"[0.8280980982342255, 2.49994293897573, 0.03889...","([[0.8280980982342255, 2.49994293897573, 0.038..."
8,19.333899,1479,2350,0,True,Optimization terminated successfully.,"[1.5888900868734286, 2.0950107429180638, 0.001...","([[1.5888900868734286, 2.0950107429180638, 0.0..."
7,23.324780,818,1364,0,True,Optimization terminated successfully.,"[2.045894068986032, 1.8664991733332439, 0.0010...","([[2.045894068986032, 1.8664991733332439, 0.00..."
5,24.055943,534,930,0,True,Optimization terminated successfully.,"[1.4005967763340004, 1.9973509961121314, 0.004...","([[1.4005967763340004, 1.9973509961121314, 0.0..."
1,24.505087,3664,5706,0,True,Optimization terminated successfully.,"[0.05, 2.4997214382703645, 0.4962733974849106,...","([[0.05, 2.4997214382703645, 0.496273397484910..."
2,25.439295,1106,1810,0,True,Optimization terminated successfully.,"[0.05, 2.4716144495584573, 0.4522552680033245,...","([[0.05, 2.4716144495584573, 0.452255268003324..."
9,29.872803,837,1405,0,True,Optimization terminated successfully.,"[2.478125583025604, 1.670149324054966, 0.00112...","([[2.478125583025604, 1.670149324054966, 0.001..."
0,21446.074219,51,189,0,True,Optimization terminated successfully.,"[0.27369591490817224, 0.1025085924734124, 0.5,...","([[0.27369591490817224, 0.1025085924734124, 0...."
3,21446.074219,54,218,0,True,Optimization terminated successfully.,"[0.3286395492055578, 0.10230539132503035, 0.23...","([[0.3286395492055578, 0.10230539132503035, 0...."
4,21446.074219,55,212,0,True,Optimization terminated successfully.,"[0.2960569090757552, 0.05, 0.4999014417624811,...","([[0.2960569090757552, 0.05, 0.499901441762481..."


#  DE fit


In [65]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))

In [66]:
fit_repeat(_c, 
           10, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [67]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
6,19.146339,850,1444,0,True,Optimization terminated successfully.,"[0.8280980982342255, 2.49994293897573, 0.03889...","([[0.8280980982342255, 2.49994293897573, 0.038..."
8,19.333899,1479,2350,0,True,Optimization terminated successfully.,"[1.5888900868734286, 2.0950107429180638, 0.001...","([[1.5888900868734286, 2.0950107429180638, 0.0..."
7,23.324780,818,1364,0,True,Optimization terminated successfully.,"[2.045894068986032, 1.8664991733332439, 0.0010...","([[2.045894068986032, 1.8664991733332439, 0.00..."
5,24.055943,534,930,0,True,Optimization terminated successfully.,"[1.4005967763340004, 1.9973509961121314, 0.004...","([[1.4005967763340004, 1.9973509961121314, 0.0..."
1,24.505087,3664,5706,0,True,Optimization terminated successfully.,"[0.05, 2.4997214382703645, 0.4962733974849106,...","([[0.05, 2.4997214382703645, 0.496273397484910..."
2,25.439295,1106,1810,0,True,Optimization terminated successfully.,"[0.05, 2.4716144495584573, 0.4522552680033245,...","([[0.05, 2.4716144495584573, 0.452255268003324..."
9,29.872803,837,1405,0,True,Optimization terminated successfully.,"[2.478125583025604, 1.670149324054966, 0.00112...","([[2.478125583025604, 1.670149324054966, 0.001..."
0,21446.074219,51,189,0,True,Optimization terminated successfully.,"[0.27369591490817224, 0.1025085924734124, 0.5,...","([[0.27369591490817224, 0.1025085924734124, 0...."
3,21446.074219,54,218,0,True,Optimization terminated successfully.,"[0.3286395492055578, 0.10230539132503035, 0.23...","([[0.3286395492055578, 0.10230539132503035, 0...."
4,21446.074219,55,212,0,True,Optimization terminated successfully.,"[0.2960569090757552, 0.05, 0.4999014417624811,...","([[0.2960569090757552, 0.05, 0.499901441762481..."
